In [141]:
import os

def GetMoves(sgf):
    tmp=''
    for i,j in enumerate(sgf):
        if j=='(':
            tmp+=GetMoves(sgf[i:])
        elif j==')':
            tmp+=j
            return tmp
        else:
            tmp+=j
    return tmp

def ReadGame(game,path='./GoDat/KGS/'):
    g=open(path+game)
    return g.read()

def StripComments(game,path='/Users/nborggren/Codes/GamePDEs/Go/GoDat/KGS/'):
    os.system('perl CommentFilter.pl '+path+game+' '+path+game.replace('.sgf','_nocom.sgf'))
    
def GetKey(sgf,key='PW'):
    a=sgf.find(key+'[')
    b=sgf[a:].find(']')
    return sgf[a+len(key)+1:a+b]  

def StripKey(sgf,key='CR'):
    while sgf.find(key+'[')>0:
        tmp=GetKey(sgf,key=key)
        sgf=sgf.replace(key+'['+tmp+']','')
    return sgf

def CleanSGF(game,path='/Users/nborggren/Codes/GamePDEs/Go/GoDat/KGS/'):
    StripComments(game)
    sgf=ReadGame(game.replace('.sgf','_nocom.sgf'))
    sgf=sgf.replace('\n','')
    for i in ['CR','WL','BL','OB','OW','C','RU','KM','TM','OT','PW','PB','WR','BR','DT','PRE','RE']:
        sgf=StripKey(sgf,key=i)
    g=open(path+game.replace('.sgf','_clean.sgf'),'w')
    g.write(sgf)
    
def GetCorners(sgf):
    c1=[]
    c2=[]
    c3=[]
    c4=[]
    moves=sgf.split(';')
    moves=moves[2:]
    locs=[i[2:4] for i in moves]
    letters='abcdefghijklmnopqrs'
    bletters='srqponmlkjihgfedcba'
    sym={}
    for i,j in zip(letters,bletters):
        sym[i]=j
    l,r=letters[0:10],letters[9:]
    for i,j in zip(moves,locs):
        if j[0] in l and j[1] in l:
            c1.append(i)
        if j[0] in l and j[1] in r:
            tmp=i.replace(j[1],sym[j[1]])
            c2.append(tmp)
        if j[0] in r and j[1] in r:
            tmp=i.replace(j[1],sym[j[1]])
            tmp=tmp.replace(j[0],sym[j[0]])
            c3.append(tmp)
        if j[0] in r and j[1] in l:
            tmp=i.replace(j[0],sym[j[0]])
            c4.append(tmp)
    return [c1,c2,c3,c4]

def WriteCorners(corner,path='./GoDat/Corners/',name='test'):
    if len(corner)==0:return
    header="(;GM[1]FF[4]CA[UTF-8]ST[2]\nSZ[19]\n;"
    record=header
    for i in corner[:-1]:record+=i+';'
    record+=corner[-1].replace(')','')+')'
    f=open(path+name+'.sgf','w')
    f.write(record)
    f.close()

def GetSyms(sgf,src='notlist'):
    c1=[]
    c2=[]
    c3=[]
    c4=[]
    c5=[]
    c6=[]
    c7=[]
    c8=[]
    if src=='notlist':
        moves=sgf.split(';')
        moves=moves[2:]
    else:
        moves=sgf
    locs=[i[2:4] for i in moves]
    letters='abcdefghijklmnopqrs'
    bletters='srqponmlkjihgfedcba'
    sym={}
    for i,j in zip(letters,bletters):
        sym[i]=j
    for i,j in zip(moves,locs):
        c1.append(i)
        tmp=j[0]+sym[j[1]]
        c2.append(i.replace(j,tmp))
        tmp=sym[j[0]]+sym[j[1]]
        c3.append(i.replace(j,tmp))
        tmp=sym[j[0]]+j[1]
        c4.append(i.replace(j,tmp))
        tmp=j[1]+j[0]
        c5.append(i.replace(j,tmp))
        tmp=j[1]+sym[j[0]]
        c6.append(i.replace(j,tmp))
        tmp=sym[j[1]]+sym[j[0]]
        c7.append(i.replace(j,tmp))
        tmp=sym[j[1]]+j[0]
        c8.append(i.replace(j,tmp))
    return [c1,c2,c3,c4,c5,c6,c7,c8]

def PatternSearch(pattern,games):
    patterns=[]
    mygames=[]
    rev = {}
    rev['B']='W'
    rev['W']='B'
    for i in games:
        try:
            corners=GetCorners(ReadGame(i.replace('.sgf','_clean.sgf')))
        except IndexError:
            continue
        for j in corners:
            a=GetSyms(j,src='list')
            for k in a:
                if k[0:3]==pattern or k[0:3]==[rev[l[0]]+l[1:] for l in pattern]:
                    mygames.append(i)
                    patterns.append(k)
    return patterns,mygames

def WritePatterns(patterns,mygames,path='./outputs/',name='test'):
    header="(;GM[1]FF[4]CA[UTF-8]ST[2]SZ[19]"
    for i,game in zip(patterns,mygames):
        if len(i)==0:continue
        tmp='(;C['+game+']'
        for j in i[:-1]:
            j=j.replace(')','')
            j=j.replace('(','')
            tmp+=j+';'
        i[-1]=i[-1].replace('(','')
        tmp+=i[-1].replace(')','')   
        tmp+=')\n'
        header+=tmp
    f=open(path+name+'.sgf','w')
    f.write(header+')')
    f.close()

f=open('./GoDat/KGS/Borggren.games')
mygames=f.read().split('\n')
games = [i for i in mygames if i.find('.sgf')>0 and i.find('/Bogoliubon-7.sgf')<0 and i.find('/Bogoliubon-6.sgf')<0 and i.find('/Bogoliubon-3.sgf')<0 and i.find('/Bogoliubon-5.sgf')<0 and i.find('/Bogoliubon-4.sgf')<0 and i.find('/Bogoliubon.sgf')<0 and i.find('/Bogoliubon-2.sgf')<0]
myana = [i for i in mygames if i.find('/Bogoliubon.sgf')>0]
print len(games),len(mygames)
games.reverse()

5086 7229


In [7]:
sgf=ReadGame(games[0])

In [31]:
for i in games:StripComments(i)

In [34]:
games[0]

'2016/2/24/Bogoliubon-tf.sgf'

In [139]:
for i in games:CleanSGF(i)

In [140]:
sgf=ReadGame(games[0].replace('.sgf','_clean.sgf'))

In [141]:
sgf

'(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]SZ[19];B[pd];W[dp];B[pp];W[dc];B[de];W[ce];B[dd];W[cd];B[ec];W[cf];B[db];W[cc];B[jc];W[nq];B[kp];W[no];B[pn];W[pr];B[qq];W[kr];B[qk];W[qf];B[qh];W[qc];B[pc];W[qd];B[pe];W[rf];B[og];W[fc];B[ed];W[fb];B[eb];W[df];B[fd];W[ke];B[ld];W[pl];B[ql];W[cn];B[fq];W[eq];B[fp];W[jq];B[mp];W[np];B[lr];W[lq];B[kq];W[mr];B[jr];W[ls];B[iq];W[dk];B[em];W[en];B[fn];W[fm];B[el];W[gm];B[dn];W[eo];B[fo];W[do];B[cm];W[dm];B[dl];W[cl];B[ek];W[bm];B[dj];W[ck];B[ej];W[im];B[ij];W[gj];B[gk];W[jp];B[jo];W[ip];B[hp];W[gn];B[io];W[hk];B[hj];W[ik];B[gi];W[ff];B[gf];W[jj];B[ji];W[ki];B[kj];W[jk];B[jh];W[kh];B[jg];W[kg];B[jf];W[kf];B[lj];W[mi];B[mj];W[ni];B[lm];W[ln];B[kn];W[km];B[kl];W[ll];B[jm];W[kk];B[ml];W[lk];B[mk];W[mm];B[km];W[nm];B[ok];W[nk];B[nl];W[pk];B[oj];W[je];B[hf];W[ph];B[pj];W[pg];B[of];W[oh];B[pb];W[qb];B[qa];W[ra];B[pa];W[cj];B[ie];W[qi];B[rj];W[qr];B[rr];W[rs];B[fr];W[om];B[pm];W[ol];B[nj];W[ri];B[qj];W[rq];B[rp];W[sr];B[dr];W[cq];B[cr];W[br];B[bs];W[ar];B[ci];

In [28]:
sgf=StripKey(sgf,key='CR')

In [61]:
sgf.count(';')

200

In [62]:
sgf=sgf.replace('\n','')

In [217]:
corners=GetCorners(ReadGame(games[0].replace('.sgf','_clean.sgf')))
WriteCorners(corners[0],name='0_0')
syms=GetSyms(ReadGame(games[0].replace('.sgf','_clean.sgf')))
for i,j in enumerate(syms):WriteCorners(syms[i],name='syms_0_'+str(i))

In [205]:
corners[0]

['W[dc]',
 'B[de]',
 'W[ce]',
 'B[dd]',
 'W[cd]',
 'B[ec]',
 'W[cf]',
 'B[db]',
 'W[cc]',
 'B[jc]',
 'W[fc]',
 'B[ed]',
 'W[fb]',
 'B[eb]',
 'W[df]',
 'B[fd]',
 'B[dj]',
 'B[ej]',
 'B[ij]',
 'W[gj]',
 'B[hj]',
 'B[gi]',
 'W[ff]',
 'B[gf]',
 'W[jj]',
 'B[ji]',
 'B[jh]',
 'B[jg]',
 'B[jf]',
 'W[je]',
 'B[hf]',
 'W[cj]',
 'B[ie]',
 'B[ci]',
 'W[bi]',
 'B[ch]',
 'W[bh]',
 'B[cb]',
 'W[jd]',
 'B[ic]',
 'B[fg]',
 'W[eg]',
 'B[eh]',
 'W[hi]',
 'B[ii]',
 'W[cg]',
 'B[dh])']

In [68]:
moves=sgf.split(';')
moves=moves[2:]

In [90]:
letters='abcdefghijklmnopqrs'
bletters='srqponmlkjihgfedcba'
sym={}
for i,j in zip(letters,bletters):
    sym[i]=j

In [76]:
l,r=letters[0:10],letters[9:]

In [161]:
c1=[]
c2=[]
c3=[]
c4=[]
for i,j in zip(moves,locs):
    if j[0] in l and j[1] in l:
        c1.append(i)
    if j[0] in l and j[1] in r:
        tmp=i.replace(j[1],sym[j[1]])
        c2.append(tmp)
    if j[0] in r and j[1] in r:
        tmp=i.replace(j[1],sym[j[1]])
        tmp=tmp.replace(j[0],sym[j[0]])
        c3.append(tmp)
    if j[0] in r and j[1] in l:
        tmp=i.replace(j[0],sym[j[0]])
        c4.append(tmp)


In [103]:
header="(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]\nRU[Japanese]SZ[19]KM[6.50]TM[600]OT[5x40 byo-yomi]\nPW[Bogoliubon]PB[tf]WR[1k]BR[1k]DT[2016-02-24]PRE[B+Resign]\n;"

In [142]:
def WriteCorners(corners,path='./GoDat/Corners/'):
    header="(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]\nRU[Japanese]SZ[19]KM[6.50]TM[600]OT[5x40 byo-yomi]\nPW[Bogoliubon]PB[tf]WR[1k]BR[1k]DT[2016-02-24]PRE[B+Resign]\n;"

In [143]:
header

'(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]\nRU[Japanese]SZ[19]KM[6.50]TM[600]OT[5x40 byo-yomi]\nPW[Bogoliubon]PB[tf]WR[1k]BR[1k]DT[2016-02-24]PRE[B+Resign]\n;'

In [112]:
f=open('test.sgf','w')
record=header
for i in c1[:-1]:record+=i+';'
record+=c1[-1].replace(')','')+')'

record+='\n'+header
for i in c2[:-1]:record+=i+';'
record+=c2[-1].replace(')','')+')'

record+='\n'+header
for i in c1[:-1]:record+=i+';'
record+=c1[-1].replace(')','')+')'

record+='\n'+header
for i in c1[:-1]:record+=i+';'
record+=c1[-1].replace(')','')+')'

f.write(record)

In [114]:
print record

(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]
RU[Japanese]SZ[19]KM[6.50]TM[600]OT[5x40 byo-yomi]
PW[Bogoliubon]PB[tf]WR[1k]BR[1k]DT[2016-02-24]PRE[B+Resign]
;W[dc];B[de];W[ce];B[dd];W[cd];B[ec];W[cf];B[db];W[cc];B[jc];W[fc];B[ed];W[fb];B[eb];W[df];B[fd];B[dj];B[ej];B[ij];W[gj];B[hj];B[gi];W[ff];B[gf];W[jj];B[ji];B[jh];B[jg];B[jf];W[je];B[hf];W[cj];B[ie];B[ci];W[bi];B[ch];W[bh];B[cb];W[jd];B[ic];B[fg];W[eg];B[eh];W[hi];B[ii];W[cg];B[dh])
(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]
RU[Japanese]SZ[19]KM[6.50]TM[600]OT[5x40 byo-yomi]
PW[Bogoliubon]PB[tf]WR[1k]BR[1k]DT[2016-02-24]PRE[B+Resign]
;W[dd];W[cf];B[fc];W[ec];B[fd];W[jc];B[jb];B[ic];W[di];B[eg];W[ef];B[ff];W[fg];B[eh];W[gg];B[df];W[ee];B[fe];W[de];B[cg];W[dg];B[dh];W[ch];B[ei];W[bg];B[dj];W[ci];B[ej];W[ig];B[ij];W[gj];B[gi];W[jd];B[je];W[id];B[hd];W[gf];B[ie];W[hi];B[hj];W[ii];W[jj];W[ji];B[jg];W[cj];B[fb];B[db];W[cc];B[cb];W[bb];B[ba];W[ab])
(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]
RU[Japanese]SZ[19]KM[6.50]TM[600]OT[5x40 byo-yomi]
PW[Bogoliubo

In [115]:
corners=[c1,c2,c3,c4]

In [119]:
joseki={}
joseki[c1[0]]={}
joseki[c1[0]][c1[1]]={}
        


In [120]:
joseki

{'W[dc]': {'B[de]': {}}}

In [121]:
fmove=set()
for i in corners:
    fmove.add(i[0])
    

In [122]:
print fmove

set(['W[dd]', 'W[dc]', 'B[dd]'])


In [124]:
sgf

'(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]RU[Japanese]SZ[19]KM[6.50]TM[600]OT[5x40 byo-yomi]PW[Bogoliubon]PB[tf]WR[1k]BR[1k]DT[2016-02-24]PRE[B+Resign];B[pd];W[dp];B[pp];W[dc];B[de];W[ce];B[dd];W[cd];B[ec];W[cf];B[db];W[cc];B[jc];W[nq];B[kp];W[no];B[pn];W[pr];B[qq];W[kr];B[qk];W[qf];B[qh];W[qc];B[pc];W[qd];B[pe];W[rf];B[og];W[fc];B[ed];W[fb];B[eb];W[df];B[fd];W[ke];B[ld];W[pl];B[ql];W[cn];B[fq];W[eq];B[fp];W[jq];B[mp];W[np];B[lr];W[lq];B[kq];W[mr];B[jr];W[ls];B[iq];W[dk];B[em];W[en];B[fn];W[fm];B[el];W[gm];B[dn];W[eo];B[fo];W[do];B[cm];W[dm];B[dl];W[cl];B[ek];W[bm];B[dj];W[ck];B[ej];W[im];B[ij];W[gj];B[gk];W[jp];B[jo];W[ip];B[hp];W[gn];B[io];W[hk];B[hj];W[ik];B[gi];W[ff];B[gf];W[jj];B[ji];W[ki];B[kj];W[jk];B[jh];W[kh];B[jg];W[kg];B[jf];W[kf];B[lj];W[mi];B[mj];W[ni];B[lm];W[ln];B[kn];W[km];B[kl];W[ll];B[jm];W[kk];B[ml];W[lk];B[mk];W[mm];B[km];W[nm];B[ok];W[nk];B[nl];W[pk];B[oj];W[je];B[hf];W[ph];B[pj];W[pg];B[of];W[oh];B[pb];W[qb];B[qa];W[ra];B[pa];W[cj];B[ie];W[qi];B[rj];W[qr];B[rr];W[rs]

In [188]:
ReadGame(games[0].replace('.sgf','_clean.sgf'))

'(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]SZ[19];B[pd];W[dp];B[pp];W[dc];B[de];W[ce];B[dd];W[cd];B[ec];W[cf];B[db];W[cc];B[jc];W[nq];B[kp];W[no];B[pn];W[pr];B[qq];W[kr];B[qk];W[qf];B[qh];W[qc];B[pc];W[qd];B[pe];W[rf];B[og];W[fc];B[ed];W[fb];B[eb];W[df];B[fd];W[ke];B[ld];W[pl];B[ql];W[cn];B[fq];W[eq];B[fp];W[jq];B[mp];W[np];B[lr];W[lq];B[kq];W[mr];B[jr];W[ls];B[iq];W[dk];B[em];W[en];B[fn];W[fm];B[el];W[gm];B[dn];W[eo];B[fo];W[do];B[cm];W[dm];B[dl];W[cl];B[ek];W[bm];B[dj];W[ck];B[ej];W[im];B[ij];W[gj];B[gk];W[jp];B[jo];W[ip];B[hp];W[gn];B[io];W[hk];B[hj];W[ik];B[gi];W[ff];B[gf];W[jj];B[ji];W[ki];B[kj];W[jk];B[jh];W[kh];B[jg];W[kg];B[jf];W[kf];B[lj];W[mi];B[mj];W[ni];B[lm];W[ln];B[kn];W[km];B[kl];W[ll];B[jm];W[kk];B[ml];W[lk];B[mk];W[mm];B[km];W[nm];B[ok];W[nk];B[nl];W[pk];B[oj];W[je];B[hf];W[ph];B[pj];W[pg];B[of];W[oh];B[pb];W[qb];B[qa];W[ra];B[pa];W[cj];B[ie];W[qi];B[rj];W[qr];B[rr];W[rs];B[fr];W[om];B[pm];W[ol];B[nj];W[ri];B[qj];W[rq];B[rp];W[sr];B[dr];W[cq];B[cr];W[br];B[bs];W[ar];B[ci];

In [8]:
corners=GetCorners(ReadGame(games[3].replace('.sgf','_clean.sgf')))

In [13]:
for j,i in enumerate(games):
    sgf=ReadGame(i.replace('.sgf','_clean.sgf'))
    handis=GetKey(sgf='HA')
    if handis in [str(k+1) for k in range(9)]:
        continue
    else:
        try:
            corners=GetCorners(sgf)
        except IndexError:
            continue
        for k,l in enumerate(corners):
            WriteCorners(l,name=str(j)+'_'+str(k))


In [184]:
games[0]

'2016/2/24/Bogoliubon-tf.sgf'

In [ ]:
f=open('test.sgf','w')
record=header
for i in c1[:-1]:record+=i+';'
record+=c1[-1].replace(')','')+')'

In [150]:
WriteCorners(corners[0])

In [178]:
sgf=ReadGame('2004/10/23/plush-Zwom.sgf')

In [179]:
sgf

'(;GM[1]FF[4]CA[UTF-8]\nRU[Japanese]SZ[13]KM[6.50]TM[1650]OT[5x30 byo-yomi]\nPW[plush]PB[Zwom]DT[2004-10-23]PC[The Kiseido Go Server (KGS) at http://kgs.kiseido.com/]RE[B+Resign]\n;B[dk]CR[dk]BL[1794.988]\n;W[dd]CR[dd]WL[1796.796]\n;B[kj]CR[kj]BL[1792.360]\n;W[jd]CR[jd]WL[1795.806]\n;B[kf]CR[kf]BL[1786.830]\n;W[ke]CR[ke]WL[1783.568]\n;B[jf]CR[jf]BL[1784.870]\n;W[ic]CR[ic]WL[1770.906]\n;B[cf]CR[cf]BL[1781.786]\n;W[ie]CR[ie]WL[1760.630]\n;B[if]CR[if]BL[1754.728]\n;W[hd]CR[hd]WL[1746.608]\n;B[cd]CR[cd]BL[1752.371]\n;W[ce]CR[ce]WL[1734.277]\n;B[de]CR[de]BL[1747.927]\n;W[be]CR[be]WL[1718.034]\n;B[bd]CR[bd]BL[1746.858]\n;W[df]CR[df]WL[1705.195]\n;B[ee]CR[ee]BL[1741.486]\n;W[bf]CR[bf]WL[1690.276]\n;B[cg]CR[cg]BL[1739.859]\n;W[cc]CR[cc]WL[1663.895]\n;B[dc]CR[dc]BL[1720.122]\n;W[ed]CR[ed]WL[1649.613]\n;B[ec]CR[ec]BL[1704.379]\n;W[fd]CR[fd]WL[1582.225]\n;B[bc]CR[bc]BL[1698.497]\n;W[ef]CR[ef]WL[1554.722]\n;B[fe]CR[fe]BL[1665.545]\n;W[ad]CR[ad]WL[1507.285]\n;B[ac]CR[ac]BL[1652.042]\n;W[af]CR[af]WL

In [23]:
corners=GetCorners(ReadGame(games[0].replace('.sgf','_clean.sgf')))

In [24]:
corners=GetCorners(ReadGame(games[3].replace('.sgf','_clean.sgf')))

In [25]:
a=GetSyms(corners[1],src='list')

In [33]:
a[4][0:3]==['W[cd]', 'B[ed]', 'W[gc]'] or a[4][0:3]==['B[cd]', 'W[ed]', 'B[gc]']

True

In [92]:
def PatternSearch(pattern,games):
    patterns=[]
    mygames=[]
    rev = {}
    rev['B']='W'
    rev['W']='B'
    for i in games:
        try:
            corners=GetCorners(ReadGame(i.replace('.sgf','_clean.sgf')))
        except IndexError:
            continue
        for j in corners:
            a=GetSyms(j,src='list')
            for k in a:
                if k[0:3]==pattern or k[0:3]==[rev[l[0]]+l[1:] for l in pattern]:
                    mygames.append(i)
                    patterns.append(k)
    return patterns,mygames

In [125]:
patterns,mygames=PatternSearch(['W[cd]', 'B[ed]', 'W[gd]'],games)

In [130]:
patterns,mygames=PatternSearch(['W[cd]', 'B[ed]', 'W[gd]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kha1h')

patterns,mygames=PatternSearch(['W[cd]', 'B[ed]', 'W[gc]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kha1l')

patterns,mygames=PatternSearch(['W[cd]', 'B[ed]', 'W[hd]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kha2h')

patterns,mygames=PatternSearch(['W[cd]', 'B[ed]', 'W[hc]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kha2l')

patterns,mygames=PatternSearch(['W[cd]', 'B[ed]', 'W[id]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kha3h')

patterns,mygames=PatternSearch(['W[cd]', 'B[ed]', 'W[ic]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kha3l')



28 28
234 234
147 147
15 15
8 8
12 12


In [133]:
patterns,mygames=PatternSearch(['W[cd]', 'B[ec]', 'W[gd]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kla1h')

patterns,mygames=PatternSearch(['W[cd]', 'B[ec]', 'W[gc]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kla1l')

patterns,mygames=PatternSearch(['W[cd]', 'B[ec]', 'W[hd]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kla2h')

patterns,mygames=PatternSearch(['W[cd]', 'B[ec]', 'W[hc]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kla2l')

patterns,mygames=PatternSearch(['W[cd]', 'B[ec]', 'W[id]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kla3h')

patterns,mygames=PatternSearch(['W[cd]', 'B[ec]', 'W[ic]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='kla3l')


31 31
145 145
83 83
58 58
8 8
28 28


In [132]:
patterns,mygames=PatternSearch(['W[dd]', 'B[fd]', 'W[hd]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hha1h')

patterns,mygames=PatternSearch(['W[dd]', 'B[fd]', 'W[hc]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hha1l')

patterns,mygames=PatternSearch(['W[dd]', 'B[fd]', 'W[id]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hha2h')

patterns,mygames=PatternSearch(['W[dd]', 'B[fd]', 'W[ic]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hha2l')

patterns,mygames=PatternSearch(['W[dd]', 'B[fc]', 'W[hd]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hla1h')

patterns,mygames=PatternSearch(['W[dd]', 'B[fc]', 'W[hc]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hla1l')

patterns,mygames=PatternSearch(['W[dd]', 'B[fc]', 'W[id]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hla2h')

patterns,mygames=PatternSearch(['W[dd]', 'B[fc]', 'W[ic]'],games)
print len(patterns),len(mygames)
WritePatterns(patterns,mygames,name='hla2l')



1 1
8 8
2 2
0 0
35 35
444 444
183 183
79 79


In [124]:
len(patterns)
WritePatterns(patterns,mygames,name='onespacehigh')

In [41]:
patterns=[]
mygames=[]
for i in games:
    try:
        corners=GetCorners(ReadGame(i.replace('.sgf','_clean.sgf')))
    except IndexError:
        continue
    for j in corners:
        a=GetSyms(j,src='list')
        for k in a:
            if k[0:3]==['W[cd]', 'B[ed]', 'W[gc]'] or k[0:3]==['B[cd]', 'W[ed]', 'B[gc]']:
                print i
                mygames.append(i)
                patterns.append(k)

2016/2/22/takyan-Bogoliubon.sgf
2016/2/22/ibara-Bogoliubon.sgf
2016/1/15/Bogoliubon-AyaBot05.sgf
2016/1/11/AyaBot6-Bogoliubon.sgf
2016/1/1/AyaMC2-Bogoliubon.sgf
2015/7/21/Bogoliubon-AyaBot02-2.sgf
2015/12/20/darkfores3-Bogoliubon.sgf
2015/12/19/MCark2-Bogoliubon.sgf
2015/12/17/Bogoliubon-AyaBot05.sgf
2014/8/17/mobi-Bogoliubon.sgf
2014/5/8/Bogoliubon-GoraUralsk.sgf
2014/5/25/Nilpferd-Bogoliubon.sgf
2014/4/22/matta-Bogoliubon.sgf
2014/3/8/Archaic-Bogoliubon.sgf
2014/2/2/mausakim-Bogoliubon.sgf
2013/9/21/jco-Bogoliubon.sgf
2013/9/15/boa-Bogoliubon.sgf
2013/9/13/Ubiquity-Bogoliubon.sgf
2013/9/13/Bogoliubon-fcwr.sgf
2013/9/12/AyaBot04-Bogoliubon.sgf
2013/8/23/clee-Bogoliubon.sgf
2013/8/10/Request-Bogoliubon.sgf
2013/8/10/Bogoliubon-6.sgf
2013/7/6/Bogoliubon-facedi.sgf
2013/6/15/asa14-Bogoliubon.sgf
2013/3/5/arnoblitz-Bogoliubon.sgf
2013/12/6/Bogoliubon-peter2.sgf
2013/12/25/Bogoliubon-Nigeteiru.sgf
2013/12/15/Prezli-Bogoliubon.sgf
2013/11/6/LDD99-Bogoliubon.sgf
2013/11/3/paulfac-Bogoliubon.

In [38]:
for i in patterns:print i

['W[cd]', 'B[ed]', 'W[gc]', 'B[cc]', 'W[dc]', 'B[dd]', 'W[ec]', 'B[fc]', 'W[fd]', 'B[fb]', 'W[cb]', 'B[bc]', 'W[eb]', 'B[gb]', 'W[ce]', 'B[ea]', 'W[db]', 'B[df]', 'W[bb]', 'B[hc]', 'W[de]', 'B[ee]', 'W[ef]', 'B[fe]', 'W[ff]', 'B[ge]', 'W[dg]', 'B[gf]', 'B[ci]', 'B[bh]', 'B[cf]', 'W[bd]', 'B[be]', 'W[ac]', 'B[cg]']
['W[cd]', 'B[ed]', 'W[gc]', 'B[cc]', 'W[cg]', 'B[dd]', 'W[bd]', 'B[eb]', 'W[jc]', 'B[bc]', 'B[ci]', 'B[eh]', 'W[ef]', 'W[cj]', 'B[ch]', 'W[dj]', 'B[ej]', 'W[eg]', 'B[id]', 'W[ic]', 'B[jd]', 'B[fe]', 'W[fh]', 'B[ei]', 'W[gf]', 'B[hd]', 'W[gd]', 'B[ge]', 'W[hh]', 'W[hf]', 'B[je]', 'W[gj]', 'W[bg]', 'B[hj]', 'B[ij]', 'B[gi]', 'W[fi]', 'B[fg]', 'W[gh]', 'B[ff]', 'W[bi]', 'B[bh]', 'W[ah]', 'B[bf]', 'W[cf]', 'B[ce]', 'W[be]', 'B[af]', 'W[de])']


In [127]:

def WritePatterns(patterns,mygames,path='./outputs/',name='test'):
    header="(;GM[1]FF[4]CA[UTF-8]ST[2]SZ[19]"
    for i,game in zip(patterns,mygames):
        if len(i)==0:continue
        tmp='(;C['+game+']'
        for j in i[:-1]:
            j=j.replace(')','')
            j=j.replace('(','')
            tmp+=j+';'
        i[-1]=i[-1].replace('(','')
        tmp+=i[-1].replace(')','')   
        tmp+=')\n'
        header+=tmp
    f=open(path+name+'.sgf','w')
    f.write(header+')')
    f.close()

In [84]:
len(patterns)

234

In [128]:
WritePatterns(patterns,mygames)

In [82]:
print patterns[47]

['B[cd]', 'W[ed]', 'B[gc]', 'W[cc]', 'B[bc]', 'W[dc]', 'B[cg]', 'W[ge]', 'B[jc](', 'W[gd]', 'B[hc]', 'B[fg]', 'W[be]', 'B[bd]', 'W[ce]', 'B[de]', 'W[dd]', 'B[cf]', 'W[df]', 'B[bf]', 'W[eg]', 'B[ae]', 'W[fh]', 'B[eh]', 'W[ei]', 'B[dh]', 'W[gg]', 'B[ff]', 'W[gf]', 'B[fi]', 'W[gh]', 'B[ef]', 'W[fj]', 'B[gi](', 'B[gj]', 'W[ej]', 'W[hj]', 'W[ij]', 'W[hi]', 'W[ci]', 'B[di]', 'W[dj]', 'B[cj]', 'W[bj]', 'W[cj]', 'B[aj]', 'W[ai]', 'B[je]', 'W[hi]', 'B[gj]', 'B[ej]', 'W[hj]', 'B[di]', 'W[hj]', 'B[ej]', 'W[be]', 'B[bd]', 'W[ce]', 'B[de]))']
